In [1]:
from datasets import load_dataset
ds = load_dataset('eth-dl-rewards/code_preference_data', split='eval')

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

MODEL = 'internlm/internlm2-7b-reward'
model = AutoModel.from_pretrained(MODEL, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='cuda')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

In [4]:
def get_reward(prompt, solution):
  messages = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": solution}
  ]
  encoded = tokenizer.apply_chat_template(messages, tokenize=False)
  inputs = tokenizer(encoded, return_tensors='pt', truncation=True, max_length=4096).to('cuda')
  with torch.no_grad():
    outputs = model(**inputs)
    reward = outputs.logits[0][0].item()
    del inputs, outputs
    return reward


In [5]:
from tqdm import tqdm
import numpy as np
import gc
correct = 0
total = 0
gaps = []
for problem, accepted, rejected in tqdm(zip(ds['problem'], ds['accepted'], ds['rejected']), total=len(ds), desc='Evaluating'):
  reward_accepted = get_reward(problem, accepted)
  reward_rejected = get_reward(problem, rejected)
  gaps.append(reward_accepted - reward_rejected)
  if reward_accepted > reward_rejected:
    correct += 1
  total += 1

  if total % 100 == 0:
    print(f"Correct: {correct}/{total} ({correct/total*100:.2f}%)")
    gc.collect()
    torch.cuda.empty_cache()

Evaluating:   2%|▏         | 101/4180 [00:15<12:12,  5.57it/s]

Correct: 94/100 (94.00%)


Evaluating:   5%|▍         | 200/4180 [00:29<12:46,  5.19it/s]

Correct: 194/200 (97.00%)


Evaluating:   7%|▋         | 300/4180 [00:53<16:22,  3.95it/s]

...

Evaluating:  96%|█████████▌| 4000/4180 [12:49<00:44,  4.04it/s]

Correct: 3868/4000 (96.70%)


Evaluating:  98%|█████████▊| 4100/4180 [13:11<00:11,  7.26it/s]

Correct: 3966/4100 (96.73%)


Evaluating: 100%|██████████| 4180/4180 [13:24<00:00,  5.19it/s]


In [6]:
print(f"Correct: {correct}/{total} ({correct/total*100:.2f}%)")

Correct: 4062/4180 (97.18%)


In [2]:
from tqdm import tqdm
import numpy as np
import gc
import torch

def get_rewards_batch(batch):
    prompts, solutions = zip(*batch)
    messages = [
        [{"role": "user", "content": prompt}, {"role": "assistant", "content": solution}]
        for prompt, solution in zip(prompts, solutions)
    ]
    encoded = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(encoded, return_tensors='pt', truncation=True, max_length=4096, padding=True).to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
        rewards = outputs.logits[:, 0].tolist()
        del inputs, outputs
        return rewards

def precompute_rewards(ds, batch_size=10):
    rewards = []
    for i in tqdm(range(0, len(ds), batch_size), desc='Precomputing Rewards'):
        batch = list(zip(ds['problem'][i:i + batch_size], ds['accepted'][i:i + batch_size], ds['rejected'][i:i + batch_size]))
        accepted_batch = [(problem, accepted) for problem, accepted, _ in batch]
        rejected_batch = [(problem, rejected) for problem, _, rejected in batch]

        reward_accepted = get_rewards_batch(accepted_batch)
        reward_rejected = get_rewards_batch(rejected_batch)

        batch_rewards = list(zip(reward_accepted, reward_rejected))
        rewards.extend(batch_rewards)

        gc.collect()
        torch.cuda.empty_cache()

    return rewards

def evaluate_from_precomputed(rewards):
    correct = 0
    total = 0
    gaps = []

    for reward_accepted, reward_rejected in rewards:
        gap = reward_accepted - reward_rejected
        gaps.append(gap)
        if reward_accepted > reward_rejected:
            correct += 1
        total += 1

    return correct, total, gaps

# Example usage:
# ds should be a list of tuples in the form [(problem, accepted, rejected), ...]
batch_size = 8
precomputed_rewards = precompute_rewards(ds, batch_size=batch_size)
correct, total, gaps = evaluate_from_precomputed(precomputed_rewards)
print(f"Final Results: Correct: {correct}/{total} ({correct/total*100:.2f}%)")


NameError: name 'ds' is not defined